In [42]:
import pandas as pd 
import nltk
import re
import os
import openpyxl
import requests
import bs4
from bs4 import BeautifulSoup

In [2]:
data=pd.read_excel('cik_list.xlsx')
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,link
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,https://www.sec.gov/Archives/edgar/data/3662/0...
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,https://www.sec.gov/Archives/edgar/data/3662/0...
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,https://www.sec.gov/Archives/edgar/data/3662/0...
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,https://www.sec.gov/Archives/edgar/data/3662/0...
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,https://www.sec.gov/Archives/edgar/data/3662/0...


In [3]:
import os

In [4]:
os.mkdir('data')
os.mkdir('data2')
os.mkdir('stop_words')

In [5]:
from time import sleep

In [6]:
links=['https://www3.nd.edu/~mcdonald/Data/ND_Stop_Words_Generic.txt','https://www3.nd.edu/~mcdonald/Data/ND_Stop_Words_Names.txt','https://www3.nd.edu/~mcdonald/Data/ND_Stop_Words_DatesandNumbers.txt','https://www3.nd.edu/~mcdonald/Data/ND_Stop_Words_Geographic.txt','https://www3.nd.edu/~mcdonald/Data/ND_Stop_Words_Currencies.txt']

In [7]:
#scrapping stop words
for i in range(len(links)):
    f=requests.get(links[i],timeout=4)
    sleep(1)
    soup=BeautifulSoup(f.text,'html.parser')
    f_obj=open('stop_words/stop_words_'+str(i)+'.txt',"w+")
    lst=re.findall('\w*[a-zA-Z]',soup.get_text().lower())
    for j in lst:
        f_obj.write(j+' ')
    f_obj.close()

In [8]:
#scraping webpages
for i in range(len(data)):
    f=requests.get(data['link'][i])
    sleep(2.5)
    text=f.text
    f_obj=open('data/'+str(i)+'.txt','w+',encoding='utf-8')
    f_obj.write(text)
    f_obj.close()

In [9]:
#checking sentences present or not
scores=[]
for i in range(len(data)):
    f_obj=open('data/'+str(i)+'.txt','r',encoding='utf-8')
    text=f_obj.read()
    f_obj.close()
    soup=BeautifulSoup(text,'html.parser')
    score=''
    page_text=soup.get_text().lower()
    if "management's discussion and analysis" in page_text:
        score+='1'
    if "quantitative and qualitative disclosures about market risk" in page_text:
        score+='2'
    if "risk factors" in page_text:
        score+='3'
    scores.append(score)

In [10]:
for i in range(len(data)):
    f_obj=open('data/'+str(i)+'.txt','r',encoding='utf-8')
    text=f_obj.read().lower()
    f_obj.close()
    soup=BeautifulSoup(text,'html.parser')
    if '1' in scores[i]:
        text1=''
        for elem in soup(text=re.compile("management's discussion and analysis")):
            text1=text1+' '+str(elem.parent)
            f_obj1=open('data2/'+str(i)+'_text1.txt','w+',encoding='utf-8')
            f_obj1.write(text1)
            f_obj1.close()
    if '2' in scores[i]:
        text2=''
        for elem in soup(text=re.compile("quantitative and qualitative disclosures about market risk")):
            text2=text2+' '+str(elem.parent)
            f_obj1=open('data2/'+str(i)+'_text2.txt','w+',encoding='utf-8')
            f_obj1.write(text2)
            f_obj1.close()
    if '3' in scores[i]:
        text3=''
        for elem in soup(text=re.compile("risk factors")):
            text3=text3+' '+str(elem.parent)
            f_obj1=open('data2/'+str(i)+'_text3.txt','w+',encoding='utf-8')
            f_obj1.write(text3)
            f_obj1.close()

In [11]:
master_dict=pd.read_excel('Dict.xlsx')
master_dict.head()

,Word,Sequence Number,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Constraining,Superfluous,Interesting,Modal,Irr_Verb,Harvard_IV,Syllables,Source
0,AARDVARK,1,81,5.690194e-09,3.068740e-09,5.779943e-07,45,0,0,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,2,1.404986e-10,8.217606e-12,7.841870e-09,1,0,0,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,8,5.619945e-10,1.686149e-10,7.096240e-08,7,0,0,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,5,3.512466e-10,1.727985e-10,7.532677e-08,5,0,0,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,1752,1.230768e-07,1.198634e-07,1.110293e-05,465,0,0,0,0,0,0,0,0,0,0,3,12of12inf


In [12]:
#making a list of stop_words
text=''
for i in range(5):
    f_obj=open('stop_words/stop_words_'+str(i)+'.txt',"r")
    text+=f_obj.read()
    f_obj.close()
    text+=' '

stop_words=re.findall('\w*[a-zA-Z]',text)    #list of stop words.


In [13]:
#making a list of positive and negative words from the dictionary
positive_words=positive_words=list(master_dict[master_dict['Positive']>0]['Word'].apply(lambda f:f.lower()))
negative_words=[]
for i in master_dict[master_dict['Negative']>0]['Word']:
    try:
        negative_words.append(i.lower())
    except:
        pass


In [14]:
number_of_words=[]#list of number of words for each documnet
words=[]#list of lists of cleaned words for each of the documents
positive_words_docs=[]#list of lists of positive words for all the documents
negative_words_docs=[]#list of lists of negative words for all the documents
#lists of positive and negative scores for all the documents
positive_scores=[]
negative_scores=[]
for i in range(len(data)):
    try:
        f_obj=open('data2/'+str(i)+'_text1.txt',"r",encoding='utf-8')
        text=f_obj.read().lower()
        f_obj.close()
        soup=BeautifulSoup(text,'html.parser')
        lst=re.findall('\w*[a-zA-Z]',soup.get_text())#list of all the words for a document
        lst_pos=[] #list of positive words for a document
        lst_neg=[] #list of negative words for a document
        lst_word=[]#list of words after cleaning i.e. after removing stop words
        for j in lst:
            if j not in stop_words:
                lst_word.append(j)
                if j in positive_words:
                    lst_pos.append(j)
                if j in negative_words:
                    lst_neg.append(j)
        positive_words_docs.append(lst_pos)
        negative_words_docs.append(lst_pos)
        positive_scores.append(len(lst_pos))
        negative_scores.append(len(lst_neg))
        words.append(lst_word)
        number_of_words.append(len(lst_word))
    
    
    except:
        positive_words_docs.append(None)
        negative_words_docs.append(None)
        positive_scores.append(None)
        negative_scores.append(None)
        words.append(None)
        number_of_words.append(None)

In [15]:
data['mda_positive_score']=positive_scores
data['mda_negative_score']=negative_scores

In [16]:
#polarity_score
polarity_scores=[]
for i in range(len(data)):
    try:
        polarity_scores.append((positive_scores[i]-negative_scores[i])/(positive_scores[i]+negative_scores[i]+0.000001))
    except:
        polarity_scores.append(None)

In [17]:
data['mda_polarity_score']=polarity_scores



In [18]:
from nltk import sent_tokenize

sentences=[]#list of lists of sentences in a given document
number_of_sentences=[]#list of number of sentencs in each of the document
for i in range(len(data)):
    try:
        f_obj=open('data2/'+str(i)+'_text1.txt',"r",encoding='utf-8')
        text=f_obj.read()
        f_obj.close()
        soup=BeautifulSoup(text,'html.parser')
        lst_sentences=sent_tokenize(soup.get_text())
        sentences.append(lst_sentences)
        number_of_sentences.append(len(lst_sentences))
    except:
        sentences.append(None)
        number_of_sentences.append(None)

In [19]:
average_sentence_lengths=[]#average sentence lengths for mda section of each document
for i in range(len(data)):
    try:
        average_sentence_lengths.append(number_of_words[i]/number_of_sentences[i])
    except:
        average_sentence_lengths.append(None)

In [20]:
data['mda_average_sentence_length']=average_sentence_lengths

In [21]:
from nltk.corpus import cmudict
d = cmudict.dict()

In [22]:
def nsyl(word):
    try:
        return [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]][0]
    except KeyError:
        return syllables(word)

def syllables(word):
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count+=1
    if count == 0:
        count +=1
    return count

In [23]:
percent_cmplx_words=[]#list of percentages of complex words 
number_of_cmplx_words=[]#list of number of complex words 
for i in range(len(data)):
    n_cmplx=0
    try:
        for j in words[i]:
            if nsyl(j)>2:
                n_cmplx+=1
        percent_cmplx_words.append(n_cmplx*100/number_of_words[i])
        number_of_cmplx_words.append(n_cmplx)
    except:
        n_cmplx=None
        percent_cmplx_words.append(n_cmplx)
        number_of_cmplx_words.append(None)

In [24]:
data['mda_percentage_of_complex_words']=percent_cmplx_words

In [25]:
fog_indexes=[] #list of fog indexes for mda section
for i in range(len(data)):
    try:
        fog_indexes.append(0.4*(percent_cmplx_words[i]+average_sentence_lengths[i]))
    except:
        fog_indexes.append(None)


In [26]:
data['mda_fog_index']=fog_indexes


In [27]:
data['mda_complex_word_count']=number_of_cmplx_words


In [28]:
data['mda_word_count']=number_of_words


In [29]:
# uncertain and constraining words
constraining=pd.read_excel('constraining_dictionary.xlsx')
uncertainty=pd.read_excel('uncertainty_dictionary.xlsx')

In [30]:
uncertain_words=[x.lower() for x in uncertainty['Word']]
constraining_words=[x.lower() for x in constraining['Word']]


In [31]:
constraining_words_docs=[] # constraining words for a given document
constraining_score=[] # number of constraining words for a given document 
uncertain_words_docs=[] #uncertain words for a given document
uncertainty_score=[] #number of uncertain words for a given document 
for i in range(len(data)):
    lst_constraining=[]
    lst_uncertainty=[]
    try:
        for j in words[i]:
            if j in constraining_words:
                lst_constraining.append(j)
            if j in uncertain_words:
                lst_uncertainty.append(j)
        constraining_words_docs.append(lst_constraining)
        constraining_score.append(len(lst_constraining))
        uncertain_words_docs.append(lst_uncertainty)
        uncertainty_score.append(len(lst_uncertainty))
    except:
        constraining_words_docs.append(None)
        constraining_score.append(None)
        uncertain_words_docs.append(None)
        uncertainty_score.append(None)

In [32]:
data['mda_uncertainty_score']=uncertainty_score
data['mda_constraining_score']=constraining_score

In [33]:
positive_word_proportions=[]
negative_word_proportions=[]
constraining_word_proportions=[]
uncertainty_word_proportions=[]
for i in range(len(data)):
    try:
        positive_word_proportions.append(positive_scores[i]/number_of_words[i])
        negative_word_proportions.append(negative_scores[i]/number_of_words[i])
        constraining_word_proportions.append(constraining_score[i]/number_of_words[i])
        uncertainty_word_proportions.append(uncertainty_score[i]/number_of_words[i])
    except:
        positive_word_proportions.append(None)
        negative_word_proportions.append(None)
        constraining_word_proportions.append(None)
        uncertainty_word_proportions.append(None)

In [34]:
data['mda_positive_word_proportion']=positive_word_proportions
data['mda_negative_word_proportion']=negative_word_proportions
data['mda_constraining_word_proportion']=constraining_word_proportions
data['mda_uncertainty_word_proportion']=uncertainty_word_proportions

In [39]:
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,link,mda_positive_score,mda_negative_score,mda_polarity_score,...,mda_percentage_of_complex_words,mda_fog_index,mda_complex_word_count,mda_word_count,mda_uncertainty_score,mda_constraining_score,mda_positive_word_proportion,mda_negative_word_proportion,mda_constraining_word_proportion,mda_uncertainty_word_proportion
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,https://www.sec.gov/Archives/edgar/data/3662/0...,181.0,521.0,-0.484330,...,48.700332,None,8506.0,17466.0,347.0,188.0,0.010363,0.029829,0.010764,0.019867
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,https://www.sec.gov/Archives/edgar/data/3662/0...,11.0,70.0,-0.728395,...,45.319905,None,765.0,1688.0,57.0,4.0,0.006517,0.041469,0.002370,0.033768
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,https://www.sec.gov/Archives/edgar/data/3662/0...,109.0,509.0,-0.647249,...,48.860459,None,5767.0,11803.0,204.0,175.0,0.009235,0.043125,0.014827,0.017284
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:


data.to_excel('final_output.xlsx',index=False)

